In [1]:
%%configure -f
{
    "conf": {
        "org.bytedeco.javacpp.maxPhysicalBytes": "0",
        "spark.jars.packages": "com.audienceproject:spark-dynamodb_2.12:1.1.1,com.johnsnowlabs.nlp:spark-nlp_2.12:3.3.4",
        "spark.pyspark.python": "python3",
        "spark.pyspark.virtualenv.enabled": "true",
        "spark.pyspark.virtualenv.type":"native",
        "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
        "spark.driver.memory":"12g"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1642937945784_0005,pyspark,idle,Link,Link,
5,application_1642937945784_0009,pyspark,shutting_down,Link,Link,


In [2]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1642937945784_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
sc.install_pypi_package("spark-nlp==3.3.4")
sc.install_pypi_package("numpy")
sc.install_pypi_package("nltk")
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/48/8c/0d83c7e606651d0bec6c8c9a03bf3acbe1d9fbf8f840aa115505222e6328/spark_nlp-3.3.4-py2.py3-none-any.whl



  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

In [4]:
from datetime import datetime
from time import sleep

import pyspark.sql.functions as f
from pyspark.ml import PipelineModel
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType
from sparknlp.annotator import * 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
path_to_model = "s3://danonki-code3/bert_clf_pipelineModel.model/"
model = PipelineModel.load(path_to_model)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
while True:
    df_tweet = (
        spark.read.option("tableName", "Tweets")
        .format("dynamodb")
        .load()
        .select(f.col("text"), f.col("extractionTime").alias("created"))
    )
    df_tweet = df_tweet.withColumn(
        "text", regexp_replace("text", r"@[A-Za-z0-9]+", "")
    ).withColumn("text", regexp_replace("text", "RT[\s:]+", ""))
    df_tweet = df_tweet.withColumn(
        "row_number",
        f.row_number().over(Window.partitionBy().orderBy(f.col("created").desc())),
    ).filter("row_number <= 10")
    df_reddit = (
        spark.read.option("tableName", "RedditPosts")
        .format("dynamodb")
        .load(
            schema=StructType(
                [
                    StructField("subreddit", StringType()),
                    StructField("created", StringType()),
                    StructField("post_body", StringType()),
                    StructField("post_id", StringType()),
                    StructField("submission_id", StringType()),
                ]
            )
        )
    )
    df_reddit = (
        df_reddit.select(f.col("post_body").alias("text"), f.col("created"))
        .withColumn(
            "row_number",
            f.row_number().over(Window.partitionBy().orderBy(f.col("created").desc())),
        )
        .filter("row_number <= 10")
    )
    df = df_tweet.union(df_reddit).select("text", "created")
    pred = model.transform(df).select("text", "created", "class")\
    .drop_duplicates(["text"])\
    .withColumn("class", f.col("class").getItem("result")[0])\
    .withColumn("processed", f.lit(str(int(datetime.now().timestamp()))))
    pred.write.mode("append").option("tableName", "bert-predictions").format("dynamodb").save()
    sleep(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…